<h1> Mercado Libre Scrapping </h1>

<h3> Librerias y requerimientos. </h3>

- Instalación de selenium y beautifulsoup4

In [1]:
#!pip install -r requirements.txt

- Instalación de driver para uso de selenium en Google Chrome.

1. Ingresar a: https://sites.google.com/chromium.org/driver/
2. Ingresar a Downloads.
3. Seleccionar la versión del driver de acuerdo a la versión de Google Chrome y el sistema operativo donde será instalado.
4. Descomprimir el archivo descargado y abrir el archivo ejecutable.
5. Extraer el path o ruta donde se encuenta el archivo chromedriver.exe

- Importar librerias.

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.keys import Keys
import pandas as pd
import warnings
from datetime import date
warnings.filterwarnings("ignore")

In [2]:
fecha = date.today().strftime('%Y-%m-%d')

<h3> 2. Mercado Libre </h3>

In [3]:
#Definir el path del chromedriver.exe
ChromePath = "C:\chromedriver.exe"

#Busqueda
search = input("- ¿Qué desea buscar?: ")

- ¿Qué desea buscar?: toyota hilux


In [4]:
#Web Driver Object
driver = webdriver.Chrome(executable_path=ChromePath)
driver.maximize_window()
driver.get("https://www.mercadolibre.com/")

In [5]:
paises = BeautifulSoup(driver.page_source, 'lxml').find_all('a', {'class': 'ml-site-link'}) #Convertir página a xml
lista_paises = pd.DataFrame(paises).reset_index().rename(columns={0:'Pais','index':'Option'})

print("- Seleccione el país donde realizará la busqueda: \n")
for i in range(len(lista_paises)):
    print(f"{i}. {lista_paises.loc[i,'Pais']}.")

option = int(input("\nOpción: "))
selection = lista_paises.loc[option,'Pais']
print(f"\n- Usted ha seleccionado: {selection}")

- Seleccione el país donde realizará la busqueda: 

0. Argentina.
1. Bolivia.
2. Brasil.
3. Chile.
4. Colombia.
5. Costa Rica.
6. Dominicana.
7. Ecuador.
8. Guatemala.
9. Honduras.
10. México.
11. Nicaragua.
12. Panamá.
13. Paraguay.
14. Perú.
15. El Salvador.
16. Uruguay.
17. Venezuela.

Opción: 0

- Usted ha seleccionado: Argentina


In [6]:
for i in paises:
    if selection in i:
        i = str(i)
        url = i.split('href="')[1].split('" id')[0] #Extraer URL del listado

driver.get(url)
time.sleep(2)

#Insertar busqueda
driver.find_element_by_class_name('nav-search-input').send_keys(search)
driver.find_element_by_class_name('nav-search-input').send_keys(Keys.RETURN)

#Paginación
time.sleep(2)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
paginas = BeautifulSoup(driver.page_source, 'html').find('li',{'class':'andes-pagination__page-count'}).get_text().replace("de ","")
paginas = BeautifulSoup(driver.page_source, 'html').find('li',{'class':'andes-pagination__page-count'}).get_text().replace("de ","")
pags = int(input(f"\n- Hay {paginas} páginas disponibles. ¿Cuantás desea consultar?: "))


- Hay 42 páginas disponibles. ¿Cuantás desea consultar?: 10


In [9]:
#Scrapping
df = pd.DataFrame(columns = ["Articulo","Año","Kilometraje","Und Monet","Precio","URL","Foto"]) #Estructura DataFrame final
pag = 1
for pag in range(1, pags+1):
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    #Página actual
    current_page = BeautifulSoup(driver.page_source, 'html').find('span',{'class':'andes-pagination__link'}).get_text()
    soup = BeautifulSoup(driver.page_source, 'html').find('section', {'class': 'ui-search-results'}) #Convertir página a xml
    soup = soup.find_all('li','ui-search-layout__item')
    
    for i in soup:
        #print(i.find('title'))
        #print(i.find("h1").get_text().strip())
        url = i.find('a')['href']
        articulo = i.find('a')['title']
        photo = i.find('div',{'class':'slick-slide slick-active'}).find('img')['src']
        und = i.find('span',{'class':'price-tag-amount'}).find('span',{'class':'price-tag-symbol'}).get_text().strip()
        price = i.find('span',{'class':'price-tag-amount'}).find('span',{'class':'price-tag-fraction'}).get_text().strip()
        attributes = i.find_all('li',{'class':'ui-search-card-attributes__attribute'})
        if len(attributes) > 1:
            aux = []
            for j in attributes:
                aux.append(j.get_text())

            year = aux[0]
            km = aux[1]

        row = pd.Series([articulo,year,km,und,price,url,photo], index = df.columns)

        #Población de DataFrame con datos de cada perfil
        df = df.append(row, ignore_index=True)
    
    print(f"- Página {current_page} cargada.")
    time.sleep(1)
    driver.find_element_by_xpath("//li[@class='andes-pagination__button andes-pagination__button--next']").click()

- Página 1 cargada.
- Página 2 cargada.
- Página 3 cargada.
- Página 4 cargada.
- Página 5 cargada.
- Página 6 cargada.
- Página 7 cargada.
- Página 8 cargada.
- Página 9 cargada.
- Página 10 cargada.


In [9]:
#Exportar Datos
df.to_excel(f"{fecha} Mercado Libre {selection} - {search} .xlsx")
print(f"- Se exportaron {len(df)} artículos.")

- Se exportaron 480 artículos.
